## Подготовка данных

В качестве корпуса я выбрала Home & Kitchen product reviews (991,794 reviews)

In [1]:
!wget 'http://snap.stanford.edu/data/amazon/Home_&_Kitchen.txt.gz'

--2020-12-19 20:16:44--  http://snap.stanford.edu/data/amazon/Home_&_Kitchen.txt.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 223371335 (213M) [application/x-gzip]
Saving to: ‘Home_&_Kitchen.txt.gz.1’

Home_&_Kitchen.txt. 100%[===================>] 213.02M  1.70MB/s    in 1m 57s  

2020-12-19 20:18:41 (1.82 MB/s) - ‘Home_&_Kitchen.txt.gz.1’ saved [223371335/223371335]



In [2]:
!pip install simplejson

In [3]:
import gzip
import simplejson

In [4]:
def parse(filename):
  f = gzip.open(filename, 'rt')
  entry = {}
  for l in f:
    l = l.strip()
    colonPos = l.find(':')
    if colonPos == -1:
      yield entry
      entry = {}
      continue
    eName = l[:colonPos]
    rest = l[colonPos+2:]
    entry[eName] = rest
  yield entry

In [5]:
data = []
for e in parse("Home_&_Kitchen.txt.gz"):
    data.append(simplejson.dumps(e))

In [6]:
import pprint

In [7]:
pprint.pprint(eval(data[204]))

{'product/price': '39.28',
 'product/productId': 'B0000630NY',
 'product/title': 'Anchor Hocking Presence Cake Dome Set',
 'review/helpfulness': '1/2',
 'review/profileName': 'Jodi "Jodi"',
 'review/score': '3.0',
 'review/summary': 'Very Pretty Cake Dome Set - Excellent Price',
 'review/text': 'Very pretty and well made cake dome set. Love the way it '
                "looks. The only thing that I don't like is that the dome it "
                'pretty heavy.',
 'review/time': '1258675200',
 'review/userId': 'A27P6BODJVI95W'}


Нормализуем текст и уберём ненужные нам поля.

Для нашей задачи можно (и даже нужно) удалить ненужные символы вроде знаков препинания

In [8]:
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [9]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))
LEMMATIZER = WordNetLemmatizer()

In [10]:
def preproc_text(text):
    text = text.lower()
    text = re.sub(REPLACE_BY_SPACE_RE, ' ', text)
    text = re.sub(BAD_SYMBOLS_RE, '', text)
    clean_text = ' '.join([word for word in text.split() if word and word not in STOPWORDS])
    tokens = list(nltk.word_tokenize(clean_text))
    lemmas = ' '.join([LEMMATIZER.lemmatize(word) for word in tokens])
    return lemmas

In [11]:
preproc_text('Anyone who buys lattes (plural) every day...')

'anyone buy latte plural every day'

Также полезно будет оставить заголовки, для удобства оставим по одному отзыву на продукт (иначе выполнение некоторых процессов займёт у нас несколько часов)

In [25]:
import pandas as pd
from tqdm import tqdm

In [ ]:
norm_texts = []
titles = []
for i, el in enumerate(data):
    pbar.write('processed: %d' %i)
    pbar.update(1)

    if 'review/text' in eval(el) and 'product/title' in eval(el):
        if eval(el)['product/title'] not in titles:
            titles.append(eval(el)['product/title'])
            norm_texts.append(preproc_text(eval(el)['review/text']))

In [31]:
norm_texts[3]

'excellent around transaction high quality item exactly described superior packaging quick shipping superb seller buy confidence pleased'

In [30]:
df = pd.DataFrame({'titles': titles, 'norm_texts': norm_texts})
df

,titles,norm_texts
0,Foamagic,anyone buy latte plural every day understand e...
1,Uniflame 4 Panel Triple-Plated Folding Firepla...,screen arrived cardboard box pretty beat scree...
2,Ariel Little Mermaid with Pearl Cake Kit,ordered cake topper june 27 2010 given estimat...
3,Umbra Centaur 36 to 54-Inch Decorative Tension...,excellent around transaction high quality item...
4,NFL Dallas Cowboys Children's Dinner Set,good quality fast delivery wonderful company p...
...,...,...
73705,Paderno World Cuisine Chromed Steel Wall Mount...,_only_ mounted attached wall contrary feature ...
73706,Black Cat Lap Square - 54 x 54 Blanket/Throw,ordered lap aquare throw blanket amazon grandm...
73707,Umbra Centaur 24 to 36-Inch Decorative Tension...,looking shower stall curtain rod found practic...
73708,Culinary Institute of America Masters Collecti...,bought nov 2010 honed yet sharpen daily use ab...


## Варианты для поиска упоминаний товаров в отзывах



1. Составление шабонов с типом товара, поиск соответствующих n-грамм и выделение названий из них.
Для реализации такого метода необходимо заранее подготовить список представленных типов товаров, например, такие данные можно попробовать выгрузить из метаданных или статистически.

    Тем не менее, несмотря на вероятное высокое качество результатов при экспериментах с различными значениям n, такой метод требует огромной предварительной работы. Более того, такой метод подойдёт для более специфичных датасетов, где сущности окажутся более свзанными и количество классов гораздо меньше, так как в раздел Home & Kitchen попадает слишком много различных товаров, начиная с пылесосов и мебели и заканчивая банными полотенцами.

2. Написание правил с помощью построения шаблонов, фиксирующих последовательность частеречных тегов и грамматической информации. Здесь опять возникает проблема с подготовкой шаблонов, требующая человческих ресурсов. Лично мне кажется, что при наличии других вариантов решения задачи лучше обойти стороной методы, использующие ручную разметку, так как здесь оказывается принципиальным человеческий фактор, делающий правила не совсем надёжными. Полученная разметка окажется достаточно субъективной, а потому, для надёности, необходима работа сразу нескольких разметчиков, что требует много времени и сил.

3. Составление мини-словаря сущностей с помощью извлечения ключевых слов + расширение словаря с помощью эмбеддингов. Такой метод подойдет для изменяющихся данных (что глобально справедливо и для нашего кейса), так как словарь можно обновлять и онлайн. Тем не менее, могут возникнуть сложности с редкими словами и названиями товаров или брендов, которых в нашем датасете должно быть много, однако давайте лучше проверим это на практике.

## Составление мини-словаря сущностей + его расширение с помощью эмбеддингов

Тексты мы уже нормализовали, займёмся выделеним ключевых слов. Так как в заголовке с большой вероятностью будет указана нужная нам сущность, пересечение заголовка и ключевых слов текста я решила брать с большим весом.

In [14]:
!pip install summa

In [15]:
from summa import keywords

In [32]:
keywords.keywords(norm_texts[1], additional_stopwords=STOPWORDS, scores=True)

[('screen', 0.37314084126473696),
 ('looking', 0.31206874068996954),
 ('looked', 0.31206874068996954),
 ('price', 0.24815122146136603),
 ('brass', 0.2472626248112969)]

In [17]:
def add_to_dict(el, d):
    if el in d:
        d[el] += 1
    else:
        d[el] = 1

In [ ]:
d = {}
for i in range(len(df)):
    pbar.write('processed: %d' %i)
    pbar.update(1)
    
    kw = keywords.keywords(df['norm_texts'][i], additional_stopwords=STOPWORDS, scores=True)
    for k in kw:
        add_to_dict(k[0], d)
        if k[0] in df['titles'][i]:
            d[k[0]] += 1

In [34]:
len(d)

72666

In [37]:
list_d = list(d.items())
list_d.sort(key=lambda i: i[1], reverse=True)


Выведем 200 самых частых ключевых слов и проанализируем их

In [38]:
list_d[:200]

[('use', 2560),
 ('great', 2420),
 ('like', 2354),
 ('look', 2309),
 ('set', 2282),
 ('product', 2179),
 ('good', 1835),
 ('make', 1786),
 ('time', 1654),
 ('pan', 1427),
 ('work', 1394),
 ('nice', 1349),
 ('year', 1261),
 ('item', 1254),
 ('price', 1247),
 ('looking', 1244),
 ('quality', 1179),
 ('little', 1164),
 ('dont', 1163),
 ('love', 1155),
 ('knife', 1122),
 ('piece', 1069),
 ('easy', 969),
 ('color', 924),
 ('im', 880),
 ('clean', 857),
 ('handle', 854),
 ('size', 828),
 ('buy', 809),
 ('purchase', 799),
 ('need', 791),
 ('perfect', 787),
 ('cooking', 747),
 ('water', 737),
 ('thing', 735),
 ('cook', 716),
 ('bought', 712),
 ('sheet', 712),
 ('amazon', 695),
 ('unit', 693),
 ('glass', 691),
 ('small', 687),
 ('fit', 686),
 ('purchased', 679),
 ('coffee', 675),
 ('pot', 674),
 ('bowl', 649),
 ('making', 646),
 ('heat', 639),
 ('ive', 627),
 ('gift', 623),
 ('lid', 596),
 ('poster', 592),
 ('want', 590),
 ('setting', 584),
 ('order', 584),
 ('cup', 566),
 ('ordered', 561),
 ('go

In [153]:
ne = ['pan', 'knife', 'sheet', 'cooking', 'glass', 'coffee', 'blanket',
      'pot', 'bowl', 'poster', 'cup', 'plate', 'plastic', 'design', 'box',
      'towel', 'bed', 'dish', 'kitchen', 'clock', 'table', 'light', 'food',
      'cleaning', 'picture', 'pillow', 'blade', 'chair', 'vacuum', 'bag',
      'mug', 'frame', 'dishwasher', 'shelf', 'christmas', 'stainless', 
      'nonstick', 'microwave', 'iron', 'spoon']

Мне кажется, получилось достаточно хорошо!
Теперь добавим каждому слову близкое к нему

In [41]:
import gensim.downloader

In [43]:
pprint.pprint(list(gensim.downloader.info()['models'].keys()))

['fasttext-wiki-news-subwords-300',
 'conceptnet-numberbatch-17-06-300',
 'word2vec-ruscorpora-300',
 'word2vec-google-news-300',
 'glove-wiki-gigaword-50',
 'glove-wiki-gigaword-100',
 'glove-wiki-gigaword-200',
 'glove-wiki-gigaword-300',
 'glove-twitter-25',
 'glove-twitter-50',
 'glove-twitter-100',
 'glove-twitter-200',
 '__testing_word2vec-matrix-synopsis']


Я думаю, что исходя из специфики жанра отзывов лучше всего взять эмбеддинги, построенные на Твиттере

In [44]:
glove_vectors = gensim.downloader.load('glove-twitter-100')

[==============================================----] 92.1% 356.7/387.1MB downloaded

In [46]:
glove_vectors.most_similar('cat')[0]

('dog', 0.875208854675293)

In [154]:
ne_emb = []
for word in ne:
    if word not in ne_emb:
        ne_emb.append(word)
        if word in glove_vectors:
            ne_emb.append(glove_vectors.most_similar(word)[0][0])

In [155]:
print(ne_emb)

['pan', 'mah', 'knife', 'knives', 'sheet', 'sheets', 'cooking', 'cooked', 'glass', 'bottle', 'coffee', 'tea', 'blanket', 'blankets', 'pot', 'pots', 'bowl', 'superbowl', 'poster', 'posters', 'cup', 'cups', 'plate', 'plates', 'plastic', 'bag', 'design', 'designer', 'box', 'bag', 'towel', 'towels', 'bed', 'sleep', 'dish', 'soap', 'kitchen', 'bathroom', 'clock', 'alarm', 'table', 'front', 'light', 'lights', 'food', 'eat', 'cleaning', 'washing', 'picture', 'pictures', 'pillow', 'pillows', 'blade', 'blades', 'chair', 'desk', 'vacuum', 'cleaner', 'mug', 'mugs', 'frame', 'fabric', 'dishwasher', 'washer', 'shelf', 'drawer', 'christmas', 'xmas', 'stainless', 'steel', 'nonstick', 'anodized', 'microwave', 'oven', 'iron', 'batman', 'spoon', 'fork']


С сожалением обнаруживаем, что близкие английские слова - это просто едиснтвенное и множественное числа.

Нам такое не очень интересно, так как тексты мы нормализовали, поэтому можно попробовать нормализовать близкое слово и сравнить с тем, которое уже лежит в `ne`, если слова совпадут, то возьмём следующее по близости. В ином случае получится, что наш нормализатор или не справляется с преобразованием, или слова действительно разные, что устраивает нас в любом случае. 

In [156]:
ne_emb = []
for word in ne:
    if word not in ne_emb:
        ne_emb.append(word)
        index = 0
        if word in glove_vectors:
            while index != -1:
                sim = glove_vectors.most_similar(word)[index][0]
                if word == preproc_text(sim):
                    index += 1
                else:
                    ne_emb.append(sim)
                    index = -1

In [157]:
NE_EMB = set(ne_emb)

In [158]:
print(NE_EMB)

{'coffee', 'shower', 'tumbler', 'cleaning', 'tea', 'cooked', 'pillow', 'xmas', 'superbowl', 'cooking', 'bed', 'chair', 'fork', 'plastic', 'bottle', 'corn', 'front', 'blanket', 'clock', 'alarm', 'dishwasher', 'poster', 'pot', 'blade', 'league', 'eat', 'designer', 'vacuum', 'washing', 'cleaner', 'stainless', 'nonstick', 'cup', 'pan', 'mah', 'drawer', 'knife', 'bag', 'sleep', 'steel', 'microwave', 'bathroom', 'soap', 'light', 'christmas', 'food', 'oven', 'batman', 'pic', 'shelf', 'towel', 'fabric', 'sheet', 'dark', 'dish', 'paper', 'burrito', 'cover', 'glass', 'spoon', 'iron', 'box', 'picture', 'frame', 'anodized', 'design', 'washer', 'kitchen', 'desk', 'plate', 'mug', 'bowl', 'table'}


Теперь совсем красота!

Соберем биграммы с полученными сущностями и дескрипторами

In [177]:
def get_bigrams(text, bigrams):
    text = text.split()
    for ne in NE_EMB:
        ind = [i for i, word in enumerate(text) if word == ne]
        if ind:
            for i in ind:
                bigrams.append(tuple(text[(i - 1):(i + 1)])) if i != 0 else None
                bigrams.append(tuple(text[i:(i + 2)])) if i != len(text) - 1 else None

In [178]:
ex_bigrams = []
ex_text = 'knife KL100 one tumbler two cup'
get_bigrams(ex_text, ex_bigrams)
ex_bigrams

[('one', 'tumbler'), ('tumbler', 'two'), ('two', 'cup'), ('knife', 'KL100')]

In [ ]:
bigrams = []
for i, text in enumerate(df['norm_texts']):
    pbar.write('processed: %d' %i)
    pbar.update(1)
    get_bigrams(text, bigrams)

In [180]:
bigrams[100:111]

[('guest', 'bowl'),
 ('bowl', 'wide'),
 ('microwave', 'dishwasher'),
 ('oven', 'microwave'),
 ('microwave', 'dishwasher'),
 ('go', 'oven'),
 ('oven', 'microwave'),
 ('love', 'dish'),
 ('dish', 'exactly'),
 ('match', 'kitchen'),
 ('kitchen', 'go')]

In [181]:
len(bigrams)

348629

## Отранжируем полученные биграммы

In [182]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder2 = nltk.collocations.BigramCollocationFinder.from_documents(bigrams)

In [183]:
finder2.apply_freq_filter(10)

In [194]:
scores_dice = finder2.score_ngrams(bigram_measures.dice)
scores_dice[:30]

[(('stainless', 'steel'), 0.43778438391069585),
 (('cast', 'iron'), 0.29029957203994294),
 (('shower', 'curtain'), 0.2494289629968022),
 (('hard', 'anodized'), 0.21296296296296297),
 (('anodized', 'aluminum'), 0.20699708454810495),
 (('american', 'league'), 0.183206106870229),
 (('dishwasher', 'safe'), 0.1497606916782461),
 (('christmas', 'gift'), 0.14928112572652188),
 (('alarm', 'clock'), 0.1487724441187248),
 (('national', 'league'), 0.14018691588785046),
 (('major', 'league'), 0.13438735177865613),
 (('vacuum', 'cleaner'), 0.12961443806398687),
 (('corn', 'cob'), 0.12448132780082988),
 (('paper', 'towel'), 0.12001823985408117),
 (('nonstick', 'surface'), 0.11161731207289294),
 (('washer', 'dryer'), 0.10829493087557604),
 (('air', 'cleaner'), 0.10597710894446799),
 (('coffee', 'maker'), 0.09923110401856956),
 (('pillow', 'case'), 0.09895833333333333),
 (('nonstick', 'coating'), 0.09825546420693804),
 (('put', 'dishwasher'), 0.09478273098017156),
 (('dinner', 'plate'), 0.081424270192

In [198]:
scores_chi = finder2.score_ngrams(bigram_measures.chi_sq)
scores_chi[:30]

[(('stainless', 'steel'), 130115.35028686954),
 (('cast', 'iron'), 109586.18951792194),
 (('shower', 'curtain'), 91440.98825294174),
 (('national', 'league'), 46829.12181160039),
 (('corn', 'cob'), 44855.670000872735),
 (('dishwasher', 'safe'), 37085.503843317965),
 (('christmas', 'gift'), 32686.662495456676),
 (('anodized', 'aluminum'), 31529.652109597555),
 (('hard', 'anodized'), 31489.021084127522),
 (('american', 'league'), 31328.147627089915),
 (('coffee', 'maker'), 30466.46261516913),
 (('nonstick', 'coating'), 26116.50293553238),
 (('fabric', 'softener'), 26091.73119803366),
 (('duvet', 'cover'), 25403.197233008603),
 (('toaster', 'oven'), 24483.096922811477),
 (('dutch', 'oven'), 23818.993248227758),
 (('pillow', 'case'), 23573.296240980428),
 (('christmas', 'present'), 22429.271435144507),
 (('nonstick', 'surface'), 20425.748505099582),
 (('tea', 'kettle'), 20267.49776694479),
 (('food', 'processor'), 18934.668930541207),
 (('major', 'league'), 18238.365793521447),
 (('washer'

In [189]:
scores_pmi = finder2.score_ngrams(bigram_measures.pmi)
scores_pmi[:30]

[(('national', 'league'), 11.608708991470102),
 (('corn', 'cob'), 10.547043718534558),
 (('american', 'league'), 10.351875924804624),
 (('major', 'league'), 10.06938915630466),
 (('ear', 'corn'), 9.719880315396775),
 (('tumbler', 'refill'), 9.107552308391195),
 (('fiber', 'washer'), 8.961565361605638),
 (('anodized', 'aluminum'), 8.79973671997233),
 (('soap', 'scum'), 8.706365352490186),
 (('washer', 'dryer'), 8.596324561967261),
 (('bridal', 'shower'), 8.445463909987957),
 (('desk', 'homework'), 8.43062222766811),
 (('hard', 'anodized'), 8.425614844537453),
 (('shower', 'curtain'), 8.3918645206039),
 (('soap', 'dispenser'), 8.207559495518748),
 (('vaccum', 'cleaner'), 8.15421311884399),
 (('fabric', 'beading'), 8.128823750327461),
 (('terible', 'washing'), 8.073153806668188),
 (('fork', 'tine'), 8.060499085234559),
 (('mild', 'soap'), 8.042762402529466),
 (('fabric', 'softener'), 8.018850908302333),
 (('blanket', 'snuggle'), 7.74399971496314),
 (('corn', 'meal'), 7.719880315396775),
 

In [190]:
from nltk.metrics.spearman import *

In [199]:
spearman_correlation(
    ranks_from_scores(scores_dice),
    ranks_from_scores(scores_chi)
)

0.46409244329538324

In [197]:
spearman_correlation(
    ranks_from_scores(scores_dice),
    ranks_from_scores(scores_pmi)
)

0.48422333349992974

In [200]:
spearman_correlation(
    ranks_from_scores(scores_chi),
    ranks_from_scores(scores_pmi)
)

0.8165304006479773

Результаты показывают, что сильной разницы между PMI и Хи-квадратом нет, а с задачей лучше всего справляется Dice. Поверхностный анализ показывет, что в Dice с меньшей вероятностью попадают биграммы, не относящиеся к наименованию товара, а нужные нам коллокации оказываются в топе. 

Однако в каждую из выборок попали случайные биграммы, что объясняется в первую очередь семантической омонимией. Так, например, слово shower, которое мы включили в наш список, имеет и побочное значение, которое приводит к *baby shower, bridal shower* в нашей выдаче.

## Группировка полученных коллокаций

In [218]:
coll_dict = {}
for ne in NE_EMB:
    coll_dict[ne] = []

In [219]:
for bigram in scores_dice:
    ne = set(list(bigram[0])) & NE_EMB
    for word in ne:
        coll_dict[word].append(' '.join(bigram[0]))

In [220]:
for ne in NE_EMB:
    coll_dict[ne] = list(set(coll_dict[ne]))

In [232]:
for ne in ['poster', 'spoon', 'soap', 'tumbler', 'coffee']:
    print(ne)
    print(coll_dict[ne][:5])

poster
['poster little', 'poster wall', 'cool poster', 'poster love', 'sized poster']
spoon
['soup spoon', 'steel spoon', 'large spoon', 'spoon slotted', 'spoon use']
soap
['use soap', 'soap holder', 'little soap', 'dishwasher soap', 'dish soap']
tumbler
['glass tumbler', 'tumbler refill', '4 tumbler', 'plastic tumbler']
coffee
['coffee time', 'tasting coffee', 'fine coffee', 'coffee cup', 'coffee use']


Получилось, что мы включили в конечную аналитику непосредственно типы товаров, как 'dishwasher soap' или 'soup spoon'. 

При анализе результатов видно, что ни в каких данных не фигурируют названия конкретных товаров и брендов, что объясняется рядом причин:
1) Можно попробовать взять большее чсило n
2) Непосредственно название товара и бренд для товаров такого типа будут фигурировать исключительно в названии. Пользователи не будут в отзыве писать ложка бренда X, более того, не станут заменять тип продукта на бренд (как это происходит, например, с айфонами). Поэтому можно ожидать, что в конкретной тематике не будет возникать проблем разрешения кореферентности, как это наоблюдается в приведённом в задании примере ("Samsung Galaxy Watch", "watch", "smartwatch").

Тем не менее, если бы эта проблемы была актуальна для моей работы, я бы попробовала выделить гиперонимы внутри тайтла. Чаще всего заголовки содержат в себе указание на тип продукта, что можно зафиксировать если не ручным созданием списка исходя из метаданных, то статистически, так как можно предположить, что чаще всего в тайтлах будут повторяться именно такие общие слова как watch, всё остальное будет встречаться сравнительно реже (однако тут необходима эвристика для порога частотности, при котором мы перестаём считать слово гиперонимом).

В результате получим разметку тайтл - гипероним. Теперь при исследовании коллокаций можно проверить каждое из слов в коллокации на входимость в тайтл (в случае неуспеха можно попробовать позаменять слова на близкие с помощью эмбеддингов). Если все слова или их синонимы окажутся в тайтле, значит, всю коллокацию можно заменить на гипероним.